In [ ]:
import json
import pandas as pd
from inseq.data import FeatureAttributionOutput

# Path to your saved JSON file
json_path = 'attributions_normal_train.json'

# Load the attribution output
out = FeatureAttributionOutput.load(json_path)

# Extract primitive dicts
dicts = out.get_scores_dicts()

# Convert source attributions to DataFrame for the first sequence
df = pd.DataFrame(dicts[0]['source_attributions'])
df.index.name = 'Generated Token'
df.columns.name = 'Input Token'
df

## Visualizations

In [ ]:
import matplotlib.pyplot as plt

# Plot heatmap of attributions
plt.figure(figsize=(10, 6))
plt.imshow(df.values, aspect='auto')
plt.xticks(range(len(df.columns)), df.columns, rotation=90)
plt.yticks(range(len(df.index)), df.index)
plt.xlabel('Input Tokens')
plt.ylabel('Generated Tokens')
plt.title('Integrated Gradients Attribution Heatmap')
plt.colorbar(label='Attribution Score')
plt.tight_layout()
plt.show()